### Leemos los datos

In [3]:
import pandas as pd 
import os
import numpy as np

ruta = "../data" 
archivo = os.path.join(ruta, "Rest-Mex_2025_train.csv") 

with open(archivo, 'r', encoding='utf-8', errors='replace') as f:
    Data = pd.read_csv(f)

#Arreglamos la mala lectura de los acentos debido a la codificación
def arregla_mojibake(texto):
    try:
        return texto.encode('latin1').decode('utf-8')
    except:
        return texto  # Si falla la conversión, deja el texto igual

Data['Title'] = Data['Title'].fillna('').apply(arregla_mojibake)
Data['Review'] = Data['Review'].fillna('').apply(arregla_mojibake)

In [2]:
print(Data['Title'][30])
print(Data['Review'][30])

El alma de Michoacán
Así como Michoacán es alma de México, Janitzio es el alma de Michoacán.Sin duda un sitio exponencial de la cultura purepecha, de verdad me imagino cuando María Félix rodaba su película de Maclovia en este sitio y me imagino aún la belleza de esa época se siga conservando.De los pueblos mágicos que existen en México sin duda este, es mi favorito.Si van a ir para día de muertos vale la pena ir a este lugar muy noche porque como se velan toda la noche, los altares están hasta media noche en su culminación.Es muy económico este tipo de turismo y lleno de riqueza cultural sin duda les encantará presumir a sus amigos en Facebook todo el colorido de este lugar.


### Limpieza del texto

In [5]:
import re
from nltk.corpus import stopwords

# Asegúrate de tener los stopwords descargados
# nltk.download('stopwords')

stopwords_es = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = texto.lower()  # Minúsculas
    texto = re.sub(r'[^a-záéíóúñü\s]', '', texto)  # Quitar puntuación y caracteres raros
    palabras = texto.split()
    palabras = [palabra for palabra in palabras if palabra not in stopwords_es]  # Remover stopwords
    return ' '.join(palabras)

Data['Texto_Limpio'] = (Data['Title'].fillna('') + ' ' + Data['Review'].fillna('')).apply(limpiar_texto)

In [3]:
Data['Texto_Limpio'] = Data['Title'].fillna('') + ' ' + Data['Review'].fillna('')

In [6]:
Data['Texto_Limpio'][10]

'gran labor gerente después años hacía años visitaba hotel descuidada limpieza mantenimiento visita fin semana cambio extraordinario gran labor gerente eugenio amplias limpias habitaciones igual salones juego piscinas mejoradas limpias temperatura agua piscina infantil excelente restaurante orange cena buffetdesayuno sabroso toque provincia local banquetes capilla extraordinarios nieta disfrutó salón juegos jungla toboganes piscina niños concurrida felicitaciones empleados actitud cortés'

## Representaciones vectoriales con mejor desempeño

### Representacion TF-IDF

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore

def Representacion_TF_IDF(dataframe):
    """
    Recibe un DataFrame con la columna 'Texto_Limpio'.
    Retorna la matriz TF-IDF.
    """
    textos = dataframe['Texto_Limpio'].fillna("").values
    
    vectorizer = TfidfVectorizer(
        max_features=2500,
        ngram_range=(1, 2),
        token_pattern=r'(?u)\b[^\d\W]+\b'
    )
    
    X_tfidf = vectorizer.fit_transform(textos)
    
    print(f"Matriz TF-IDF generada con forma: {X_tfidf.shape}")
    return X_tfidf, vectorizer

### Representacion Word2Vec

In [12]:
from gensim.models import Word2Vec
def Representacion_word2vec(dataframe, vector_size=100, window=5, min_count=5, sg=1, epochs=10):
    """
    Entrena Word2Vec y obtiene el vector promedio de cada documento.
    - sg=1 usa skip-gram (mejor para semántica)
    - min_count=5 ignora palabras raras
    """
    textos = dataframe['Texto_Limpio'].fillna("").apply(str.split).values

    # Entrenar el modelo Word2Vec
    model = Word2Vec(
        sentences=textos, 
        vector_size=vector_size, 
        window=window, 
        min_count=min_count, 
        sg=sg,  # 1=skip-gram, 0=CBOW
        workers=4,
        epochs=epochs
    )

    # Crear el vector promedio para cada documento
    def obtener_vector_promedio(tokens):
        vectores = [model.wv[word] for word in tokens if word in model.wv]
        return np.mean(vectores, axis=0) if vectores else np.zeros(model.vector_size)

    vectores_documento = np.array([obtener_vector_promedio(tokens) for tokens in textos])
    
    return vectores_documento, model


### Representacion FastText

In [13]:
from gensim.models import FastText
def Representacion_fasttext(dataframe, vector_size=100, window=5, min_count=5, sg=1, epochs=10):
    """
    Entrena FastText y obtiene el vector promedio por documento.
    - sg=1 usa skip-gram (mejor semántica)
    - min_count=5 ignora palabras raras
    """
    textos = dataframe['Texto_Limpio'].fillna("").apply(str.split).values

    # Entrenar el modelo FastText
    model = FastText(
        sentences=textos,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,  # 1=skip-gram, 0=CBOW
        workers=4,
        epochs=epochs
    )

    # Crear el vector promedio para cada documento
    def obtener_vector_promedio(tokens):
        vectores = [model.wv[word] for word in tokens if word in model.wv]
        return np.mean(vectores, axis=0) if vectores else np.zeros(model.vector_size)

    vectores_documento = np.array([obtener_vector_promedio(tokens) for tokens in textos])

    print(f"Matriz FastText generada con forma: {vectores_documento.shape}")
    return vectores_documento, model


### Aplicamos las representaciones con parametros especificos

In [37]:
X_tfidf, vectorizer = Representacion_TF_IDF(Data)

Matriz TF-IDF generada con forma: (208051, 2500)


In [21]:
#X_word2vec, modelow2v =  Representacion_word2vec(Data, vector_size=100, window=2, min_count=3, sg=1, epochs=10)
X_word2vec, modelow2v =  Representacion_word2vec(Data, vector_size=100, window=3, min_count=5, sg=1, epochs=10)

In [40]:
X_fasttext, modelo_fasttext = Representacion_fasttext(Data, vector_size=100, window=2, min_count=3, sg=1, epochs=10)

Matriz FastText generada con forma: (208051, 100)


## Algoritmos de Aprendizaje Supervisado

### 1. Regresion Logistica Multiclase

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

def modelo_logistica(X, y, test_size=0.2, random_state=42):
    """
    Entrena un modelo de regresión logística multiclase.
    """
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Crear el modelo de regresión logística
    log_reg = LogisticRegression(max_iter=1000, solver='liblinear', random_state=random_state)
    

    # Usar OneVsRestClassifier para clasificación multiclase
    modelo = OneVsRestClassifier(log_reg)

    # Ajustar el modelo
    modelo.fit(X_train, y_train)  # Aquí cambias X_tfidf por X_train

    # Hacer predicciones
    y_pred = modelo.predict(X_test)

    # Evaluar el desempeño
    print("Clasificación:\n", classification_report(y_test, y_pred))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
    
    return modelo


### 2. Random Forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def modelo_random_forest(X, y, test_size=0.2, random_state=42, n_estimators=100):
    """
    Entrena un modelo de Random Forest multiclase.
    """
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Crear el modelo Random Forest
    model = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Hacer predicciones
    y_pred = model.predict(X_test)

    # Evaluar el desempeño
    print("Clasificación:\n", classification_report(y_test, y_pred))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

    return model


### Ejecutamos...

### Variable 'Type'

In [38]:
modelo_logistica(X_tfidf, Data['Type'])

Clasificación:
               precision    recall  f1-score   support

  Attractive       0.96      0.96      0.96     14000
       Hotel       0.95      0.91      0.93     10305
  Restaurant       0.94      0.96      0.95     17306

    accuracy                           0.95     41611
   macro avg       0.95      0.95      0.95     41611
weighted avg       0.95      0.95      0.95     41611

Matriz de confusión:
 [[13441   188   371]
 [  262  9406   637]
 [  371   256 16679]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

In [21]:
modelo_logistica(X_word2vec, Data['Type'])

NameError: name 'X_word2vec' is not defined

In [46]:
modelo_logistica(X_fasttext, Data['Type'])

Clasificación:
               precision    recall  f1-score   support

  Attractive       0.95      0.96      0.95     14000
       Hotel       0.93      0.90      0.92     10305
  Restaurant       0.94      0.96      0.95     17306

    accuracy                           0.94     41611
   macro avg       0.94      0.94      0.94     41611
weighted avg       0.94      0.94      0.94     41611

Matriz de confusión:
 [[13376   299   325]
 [  333  9306   666]
 [  305   371 16630]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

### Variable 'Polarity'

In [39]:
modelo_logistica(X_tfidf, Data['Polarity'])

Clasificación:
               precision    recall  f1-score   support

         1.0       0.64      0.51      0.57      1063
         2.0       0.42      0.09      0.15      1106
         3.0       0.50      0.27      0.35      3059
         4.0       0.50      0.31      0.38      9272
         5.0       0.77      0.94      0.85     27111

    accuracy                           0.72     41611
   macro avg       0.57      0.42      0.46     41611
weighted avg       0.68      0.72      0.68     41611

Matriz de confusión:
 [[  541    75   167    84   196]
 [  200    97   335   196   278]
 [   74    47   840  1099   999]
 [   18     7   271  2878  6098]
 [    7     3    75  1482 25544]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

In [44]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_word2vec = scaler.fit_transform(X_word2vec)
modelo_logistica(X_word2vec, Data['Polarity'])

Clasificación:
               precision    recall  f1-score   support

         1.0       0.60      0.57      0.59      1063
         2.0       0.36      0.01      0.02      1106
         3.0       0.44      0.21      0.28      3059
         4.0       0.46      0.25      0.32      9272
         5.0       0.76      0.95      0.84     27111

    accuracy                           0.70     41611
   macro avg       0.52      0.40      0.41     41611
weighted avg       0.65      0.70      0.66     41611

Matriz de confusión:
 [[  609    16   169   102   167]
 [  246    14   313   240   293]
 [  105     9   638  1120  1187]
 [   34     0   249  2280  6709]
 [   16     0    82  1253 25760]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

In [43]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_fasttext = scaler.fit_transform(X_fasttext)

modelo_logistica(X_fasttext, Data['Polarity'])

Clasificación:
               precision    recall  f1-score   support

         1.0       0.60      0.56      0.58      1063
         2.0       0.35      0.01      0.02      1106
         3.0       0.45      0.21      0.28      3059
         4.0       0.47      0.26      0.33      9272
         5.0       0.76      0.95      0.84     27111

    accuracy                           0.71     41611
   macro avg       0.53      0.40      0.41     41611
weighted avg       0.65      0.71      0.66     41611

Matriz de confusión:
 [[  598    15   160    91   199]
 [  247    13   317   238   291]
 [   98     8   636  1119  1198]
 [   30     0   223  2416  6603]
 [   17     1    84  1309 25700]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

### Aplicando Random Forest

In [ ]:
modelo_random_forest(X_tfidf, Data['Polarity'])

### Veamos que pasa si reducimos su dimensionalidad con PCA

In [24]:
from sklearn.decomposition import PCA
def aplicar_PCA(X, n_componentes=3):
    """
    Aplica PCA para reducir la dimensionalidad de las representaciones a n_componentes (por defecto 3).
    X: Matriz de características (puede ser X_tfidf, X_word2vec, X_fastext)
    n_componentes: Número de dimensiones al que se quiere reducir
    Retorna: Matriz reducida a n_componentes dimensiones
    """
    # Inicializar PCA con el número deseado de componentes
    pca = PCA(n_components=n_componentes)

    # Ajustar y transformar los datos
    X_pca = pca.fit_transform(X)

    # Mostrar la varianza explicada por cada componente (opcional)
    print(f"Varianza explicada por cada componente: {pca.explained_variance_ratio_}")
    print(f"Varianza total explicada: {np.sum(pca.explained_variance_ratio_):.2f}")

    return X_pca


In [25]:
X_tfidf_pca = aplicar_PCA(X_tfidf, n_componentes=3)
X_word2vec_pca = aplicar_PCA(X_word2vec, n_componentes=3)
#X_fastext_pca = aplicar_PCA(X_fasttext, n_componentes=3)


Varianza explicada por cada componente: [0.01069767 0.00682018 0.00560145]
Varianza total explicada: 0.02
Varianza explicada por cada componente: [0.19979849 0.08476903 0.06562348]
Varianza total explicada: 0.35


In [26]:
modelo_logistica(X_tfidf_pca, Data['Type'])

Clasificación:
               precision    recall  f1-score   support

  Attractive       0.87      0.91      0.89     14000
       Hotel       0.90      0.79      0.84     10305
  Restaurant       0.87      0.91      0.89     17306

    accuracy                           0.88     41611
   macro avg       0.88      0.87      0.87     41611
weighted avg       0.88      0.88      0.88     41611

Matriz de confusión:
 [[12782   351   867]
 [  795  8119  1391]
 [ 1037   593 15676]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

In [27]:
modelo_logistica(X_word2vec_pca, Data['Type'])

Clasificación:
               precision    recall  f1-score   support

  Attractive       0.92      0.92      0.92     14000
       Hotel       0.87      0.84      0.85     10305
  Restaurant       0.91      0.93      0.92     17306

    accuracy                           0.90     41611
   macro avg       0.90      0.89      0.90     41611
weighted avg       0.90      0.90      0.90     41611

Matriz de confusión:
 [[12823   651   526]
 [  630  8633  1042]
 [  505   691 16110]]


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

In [28]:
modelo_logistica(X_tfidf_pca, Data['Polarity'])

Clasificación:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1063
         2.0       0.00      0.00      0.00      1106
         3.0       0.00      0.00      0.00      3059
         4.0       0.30      0.00      0.00      9272
         5.0       0.65      1.00      0.79     27111

    accuracy                           0.65     41611
   macro avg       0.19      0.20      0.16     41611
weighted avg       0.49      0.65      0.51     41611

Matriz de confusión:
 [[    0     0     0     0  1063]
 [    0     0     0     0  1106]
 [    0     0     0     1  3058]
 [    0     0     0     3  9269]
 [    0     0     0     6 27105]]


c:\Users\uzgre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\uzgre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\uzgre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

In [29]:
modelo_logistica(X_word2vec_pca, Data['Polarity'])

Clasificación:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00      1063
         2.0       0.00      0.00      0.00      1106
         3.0       0.00      0.00      0.00      3059
         4.0       0.00      0.00      0.00      9272
         5.0       0.65      1.00      0.79     27111

    accuracy                           0.65     41611
   macro avg       0.13      0.20      0.16     41611
weighted avg       0.42      0.65      0.51     41611

Matriz de confusión:
 [[    0     0     0     0  1063]
 [    0     0     0     0  1106]
 [    0     0     0     0  3059]
 [    0     0     0     0  9272]
 [    0     0     0     0 27111]]


c:\Users\uzgre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\uzgre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\uzgre\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1000, random_state=42,
                                                 solver='liblinear'))

### Redes neuronales

In [64]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

def modelo_multilayer_perceptron(X, y, test_size=0.2, random_state=42):
    """
    Entrena un perceptrón multicapa (MLP) usando Scikit-Learn.
    """
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    # Definir el modelo
    mlp = MLPClassifier(
        hidden_layer_sizes=(256, 128, 64),
        activation='relu',
        solver='adam',
        alpha=1e-4,
        learning_rate='adaptive',
        max_iter=300,
        random_state=random_state,
        early_stopping=True
    )

    # Entrenar el modelo
    mlp.fit(X_train, y_train)

    # Predicciones
    y_pred = mlp.predict(X_test)

    # Evaluación
    print("Clasificación:\n", classification_report(y_test, y_pred, zero_division=0))
    print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

    return mlp



In [65]:
modelo_multilayer_perceptron(X_word2vec, Data['Type'])

Clasificación:
               precision    recall  f1-score   support

  Attractive       0.97      0.96      0.97     14000
       Hotel       0.94      0.94      0.94     10305
  Restaurant       0.95      0.97      0.96     17306

    accuracy                           0.96     41611
   macro avg       0.96      0.95      0.95     41611
weighted avg       0.96      0.96      0.96     41611

Matriz de confusión:
 [[13422   248   330]
 [  148  9647   510]
 [  210   351 16745]]


MLPClassifier(early_stopping=True, hidden_layer_sizes=(256, 128, 64),
              learning_rate='adaptive', max_iter=300, random_state=42)

In [67]:
modelo_multilayer_perceptron(X_word2vec, Data['Polarity'])

Clasificación:
               precision    recall  f1-score   support

         1.0       0.56      0.61      0.58      1063
         2.0       0.36      0.18      0.24      1106
         3.0       0.43      0.34      0.38      3059
         4.0       0.50      0.25      0.34      9272
         5.0       0.78      0.94      0.85     27111

    accuracy                           0.72     41611
   macro avg       0.53      0.47      0.48     41611
weighted avg       0.67      0.72      0.68     41611

Matriz de confusión:
 [[  653   145   172    26    67]
 [  282   204   411    94   115]
 [  139   163  1047   901   809]
 [   58    44   590  2356  6224]
 [   38    16   205  1331 25521]]


MLPClassifier(early_stopping=True, hidden_layer_sizes=(256, 128, 64),
              learning_rate='adaptive', max_iter=300, random_state=42)

In [66]:
modelo_multilayer_perceptron(X_word2vec, Data['Town'])

Clasificación:
                             precision    recall  f1-score   support

                    Ajijic       0.60      0.42      0.50       746
                   Atlixco       0.30      0.35      0.32       288
                   Bacalar       0.54      0.58      0.56      2197
                    Bernal       0.39      0.26      0.32       246
           Chiapa_de_Corzo       0.66      0.61      0.63       194
                   Cholula       0.49      0.42      0.45       563
                  Coatepec       0.47      0.30      0.37       147
                     Creel       0.53      0.34      0.41       386
           Cuatro_Cienegas       0.39      0.33      0.36       170
                 Cuetzalan       0.39      0.06      0.10       200
           Dolores_Hidalgo       0.66      0.29      0.41       164
          Huasca_de_Ocampo       0.68      0.35      0.46       320
              Isla_Mujeres       0.78      0.74      0.76      5993
         Ixtapan_de_la_Sal     

MLPClassifier(early_stopping=True, hidden_layer_sizes=(256, 128, 64),
              learning_rate='adaptive', max_iter=300, random_state=42)